<a href="https://colab.research.google.com/github/Infinitylogo/Machine_learning_projects/blob/main/LLM_FINE_TUNE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# REF :-- https://www.datacamp.com/tutorial/fine-tuning-large-language-models



In [ ]:
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 8.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [ ]:
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 9.6 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed

In [ ]:
!pip install datasets

In [ ]:
from datasets import load_dataset
import pandas as pd

In [ ]:
dataset = load_dataset("mteb/tweet_sentiment_extraction")
df = pd.DataFrame(dataset['train'])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/27481 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3534 [00:00<?, ? examples/s]

In [ ]:
list(df['text'])[:50]

[' I`d have responded, if I were going',
 ' Sooo SAD I will miss you here in San Diego!!!',
 'my boss is bullying me...',
 ' what interview! leave me alone',
 ' Sons of ****, why couldn`t they put them on the releases we already bought',
 'http://www.dothebouncy.com/smf - some shameless plugging for the best Rangers forum on earth',
 '2am feedings for the baby are fun when he is all smiles and coos',
 'Soooo high',
 ' Both of you',
 ' Journey!? Wow... u just became cooler.  hehe... (is that possible!?)',
 ' as much as i love to be hopeful, i reckon the chances are minimal =P i`m never gonna get my cake and stuff',
 'I really really like the song Love Story by Taylor Swift',
 'My Sharpie is running DANGERously low on ink',
 'i want to go to music tonight but i lost my voice.',
 'test test from the LG enV2',
 'Uh oh, I am sunburned',
 ' S`ok, trying to plot alternatives as we speak *sigh*',
 'i`ve been sick for the past few days  and thus, my hair looks wierd.  if i didnt have a hat on i

In [ ]:
numlabels = len(df['label_text'].unique())

In [ ]:
from transformers import GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

def tokenize_function(examples):
   return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

Map:   0%|          | 0/27481 [00:00<?, ? examples/s]

Map:   0%|          | 0/3534 [00:00<?, ? examples/s]

In [ ]:
#data after tokenization (we got label_text, input_ids and attention masks)
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'text', 'label', 'label_text', 'input_ids', 'attention_mask'],
        num_rows: 27481
    })
    test: Dataset({
        features: ['id', 'text', 'label', 'label_text', 'input_ids', 'attention_mask'],
        num_rows: 3534
    })
})

In [ ]:
tokenized_datasets['train']['label_text'][:5]

['neutral', 'negative', 'negative', 'negative', 'negative']

In [ ]:
training_dataset = tokenized_datasets['train'].shuffle(seed=42).select(range(1000))
testing_dataset = tokenized_datasets['test'].shuffle(seed=42).select(range(1000))

In [ ]:
#initialize based models
from transformers import GPT2ForSequenceClassification

In [ ]:
base_model = GPT2ForSequenceClassification.from_pretrained("gpt2", num_labels=numlabels)

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# code for model evaluate

import evaluate

In [ ]:
metric = evaluate.load("accuracy")
import numpy as np
def metric_accuracy(pred):
  logits, labels = pred
  predictions = np.argmax(logits, axis= -1)
  return metric.compute(predictions = predictions, references= labels)

Fine-tune using the Trainer Method


In [ ]:
# lets fine tune the model using the custom data created and using Trainer
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir = "./model_results",
    per_device_train_batch_size = 1,
    per_device_eval_batch_size = 1,
    # num_train_epochs = 1,
    # evaluation_strategy = "epoch",
    # save_strategy = "epoch",
    # learning_rate = 2e-5
    gradient_accumulation_steps= 4
    )

trainer = Trainer(
    model = base_model,
    args = training_args,
    train_dataset = training_dataset,
    eval_dataset= testing_dataset,
    compute_metrics = metric_accuracy,
)

In [17]:
trainer.train()

Step,Training Loss


Step,Training Loss
500,0.989800


TrainOutput(global_step=750, training_loss=0.8364325561523438, metrics={'train_runtime': 813.3497, 'train_samples_per_second': 3.688, 'train_steps_per_second': 0.922, 'total_flos': 1567794659328000.0, 'train_loss': 0.8364325561523438, 'epoch': 3.0})

In [18]:
trainer.evaluate()

{'eval_loss': 0.9099463820457458,
 'eval_accuracy': 0.745,
 'eval_runtime': 92.3926,
 'eval_samples_per_second': 10.823,
 'eval_steps_per_second': 10.823,
 'epoch': 3.0}

In [33]:
trainer.save_model("./modelResults")
tokenizer.save_pretrained("./modelResults")

('./modelResults/tokenizer_config.json',
 './modelResults/special_tokens_map.json',
 './modelResults/vocab.json',
 './modelResults/merges.txt',
 './modelResults/added_tokens.json')

In [34]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments
import torch

# Define the path to the saved model directory
model_path = "./modelResults/"

# Load the saved model and tokenizer
model = AutoModelForSequenceClassification.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Example input text data for sentiment analysis
texts = ["I love this!", "I hate this!"]

# Tokenize the input data
inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")

In [32]:
# !rm -rf modelResults

In [36]:
# Example test data
from datasets import Dataset
test_texts = ["I love this!", "I hate this!"]
test_data = {"text": test_texts}

# Create a Dataset object
test_dataset = Dataset.from_dict(test_data)

# Tokenize the test data
tokenized_test_dataset = test_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

In [37]:
predictions = trainer.predict(tokenized_test_dataset)

In [39]:
print(predictions.predictions)

[[-2.6685581  -0.90610313  6.5517755 ]
 [ 1.7988725  -5.76186    -7.208847  ]]


In [40]:
# Print the predictions
# If the model outputs logits, you may want to convert them to probabilities
probs = torch.nn.functional.softmax(torch.tensor(predictions.predictions),dim=-1)

# Print the predicted probabilities
print(probs)

tensor([[9.8939e-05, 5.7649e-04, 9.9932e-01],
        [9.9936e-01, 5.2016e-04, 1.2238e-04]])


In [41]:
# Optionally, get the predicted class labels
predicted_class_labels = torch.argmax(probs, dim=1)

# Print the predicted class labels
print(predicted_class_labels)

tensor([2, 0])




*   1 neutral
*   0 negative
*   2 postive




In [49]:
# list(zip(tokenized_datasets['train']['label'], tokenized_datasets['train']['label_text']))

[(1, 'neutral'),
 (0, 'negative'),
 (0, 'negative'),
 (0, 'negative'),
 (0, 'negative'),
 (1, 'neutral'),
 (2, 'positive'),
 (1, 'neutral'),
 (1, 'neutral'),
 (2, 'positive'),
 (1, 'neutral'),
 (2, 'positive'),
 (0, 'negative'),
 (0, 'negative'),
 (1, 'neutral'),
 (0, 'negative'),
 (0, 'negative'),
 (0, 'negative'),
 (0, 'negative'),
 (1, 'neutral'),
 (1, 'neutral'),
 (2, 'positive'),
 (1, 'neutral'),
 (1, 'neutral'),
 (1, 'neutral'),
 (2, 'positive'),
 (0, 'negative'),
 (0, 'negative'),
 (2, 'positive'),
 (0, 'negative'),
 (2, 'positive'),
 (2, 'positive'),
 (0, 'negative'),
 (2, 'positive'),
 (1, 'neutral'),
 (1, 'neutral'),
 (0, 'negative'),
 (1, 'neutral'),
 (0, 'negative'),
 (2, 'positive'),
 (1, 'neutral'),
 (2, 'positive'),
 (1, 'neutral'),
 (1, 'neutral'),
 (2, 'positive'),
 (1, 'neutral'),
 (0, 'negative'),
 (1, 'neutral'),
 (0, 'negative'),
 (0, 'negative'),
 (1, 'neutral'),
 (1, 'neutral'),
 (1, 'neutral'),
 (0, 'negative'),
 (2, 'positive'),
 (1, 'neutral'),
 (0, 'negative'